# Optimization-based economic models – an introduction

Main topics

- The toolkit
- 
- Utility maximization
- Profit maximization

## Optimization in Julia

- Julia has a number of packages that solve various classes of optimization problems
    + [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) (more generally, see [JuliaNLSolvers](https://github.com/JuliaNLSolvers))
    + [Optimization.jl](https://github.com/SciML/Optimization.jl)
    + [BlackBoxOptim.jl](https://github.com/robertfeldt/BlackBoxOptim.jl)
- Some of these packages offer different and complementary approaches to the same type of problem
- The diversity of packages results to a proliferation of different APIs for defining and solving problems $\Rightarrow$ there is a need for a unified interface that facilitates access to the different packages
- [JuMP](https://jump.dev/) offers a domain-specific language for defining and working with optimization problems

To install JuMP and a couple of solvers, run the following:

In [ ]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Ipopt")
Pkg.add("HiGHS")

$$ \min_{x_1,x_2} ~ (x_1-3)^2 + (x_2-2)^2 $$
s.t.
$$x_1^2+x_2^2 \leq 5$$
$$ x_1+x_2 \leq 3 $$
$$ x_1,x_2 \geq 0 $$


In [29]:
using JuMP, Ipopt

In [19]:
model = nothing

In [45]:
model = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [46]:
@variable(model, x₁ >= 0)

x₁

In [47]:
@variable(model, x₂ >= 0)

x₂

In [48]:
@objective(model,Min,(x₁-3)^2+(x₂-2)^2)

x₁² + x₂² - 6 x₁ - 4 x₂ + 13

In [49]:
@constraint(model, c₁, x₁^2 + x₂^2 <= 5)

c₁ : x₁² + x₂² <= 5

In [50]:
@constraint(model, c₂,x₁ + x₂ <= 3)

c₂ : x₁ + x₂ <= 3

In [51]:
print(model)

In [52]:
optimize!(model)

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2900200e+01 0.00e+00 3.31e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

The output is quite detailed in this case but in principle it is good practice to check if model is solved and a solution found:

In [53]:
is_solved_and_feasible(model)

true

In [54]:
termination_status(model)

LOCALLY_SOLVED::TerminationStatusCode = 4

`termination_status` provides the reason the solver stopped, i.e. did it succeed, fail etc. [List of termination statuses](https://jump.dev/JuMP.jl/stable/moi/reference/models/#MathOptInterface.TerminationStatusCode)

In [55]:
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

In [56]:
dual_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

`primal_status` and `dual_status` tell us how to interpret the result vector. [List of result statuses](https://jump.dev/JuMP.jl/stable/moi/reference/models/#MathOptInterface.ResultStatusCode)

In [57]:
value(x₁)

1.9999668104799597

In [58]:
value(x₂)

1.0000332190655201

In [60]:
shadow_price(c₁)

-6.640927005848035e-5

In [61]:
shadow_price(c₂)

-1.9998007433241507

## Profit maximization



## Consumer choice

In [90]:
p = [1,2]
I = 20
a1 = 0.2
a2 = 1-a1
umod = Model(Ipopt.Optimizer)
set_silent(umod)
@variable(umod, x[1:2] >= 0)
@constraint(umod, budget, p'*x == I)
@objective(umod, Max, x[1]^a1*x[2]^a2)
# @objective(umod, obj2, a1*log(x[1]) + a2*log(x[2]))
optimize!(umod)
cons1, cons2 = value.(x);
println("Consumption of good 1 = $cons1, Consumption of good 2 = $cons2")

@objective(umod, Max, a1*log(x[1]) + a2*log(x[2]))
optimize!(umod)
cons1, cons2 = value.(x);
println("Consumption of good 1 = $cons1, Consumption of good 2 = $cons2")

Consumption of good 1 = 4.000000004319392, Consumption of good 2 = 7.9999999978403045
Consumption of good 1 = 4.000176118926564, Consumption of good 2 = 7.999911940536717
